<a href="https://colab.research.google.com/github/martaarozarena/KSchool-Master-Final-Project/blob/master/exog_varaibles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
sheets=["c1_schoolclosing","c2_workplaceclosing","c3_cancelpublicevents","c4_restrictionsongatherings","c5_closepublictransport","c6_stayathomerequirements","c7_domestictravel","c8_internationaltravel","h1_publicinfocampaign","h2_testingpolicy","h3_contacttracing"]
initialdate="01-01-2020"
#enddate="30-06-2020"

In [3]:
var = pd.read_excel("https://github.com/OxCGRT/covid-policy-tracker/raw/master/data/timeseries/OxCGRT_timeseries_all.xlsx",sheet_name=sheets)

In [4]:
j=1
for i in var.keys():
  var[i]=var[i].drop("CountryCode",axis=1)
  var[i].index=var[i].CountryName
  var[i]=var[i].drop("CountryName",axis=1)
  var[i]=var[i][:-3]
  var[i]=var[i].T
  var[i].index=pd.to_datetime(var[i].index,format="%d%b%Y")
#  var[i]=var[i].truncate(initialdate,enddate)
  j=j+1

In [5]:
b=1
for i in sheets:
  if b==1: 
    var[i].columns="{}_".format(i[3:])+var[i].columns
    exogenas=var[i]
  else:
    concat=var[i]
    concat.columns="{}_".format(i[3:])+var[i].columns
    exogenas=pd.concat([exogenas,concat],axis=1)
  b=b+1

In [6]:
#download the mask wearing data

In [7]:
url2 = "https://drive.google.com/file/d/1vbPOY0hF-Dox_HONp-brgf36TST2vs49/view?usp=sharing"
path2 = 'https://drive.google.com/uc?export=download&id='+url2.split('/')[-2]
var_percent_mask = pd.read_csv(path2,sep=";")

In [8]:
var_percent_mask.DateTime=pd.to_datetime(var_percent_mask.DateTime)
var_percent_mask.index=var_percent_mask.DateTime
var_percent_mask.drop("DateTime",axis=1,inplace=True)

In [9]:
#add the missing dates
var_percent_mask=var_percent_mask.resample("D").mean()

In [10]:
#fill the missing dates with an interpolation
var_percent_mask.interpolate(method='linear',inplace=True)
var_percent_mask.fillna(method="bfill",inplace=True)


In [11]:
#rename the variable to be added in the general dataframe and choose the dates
var_percent_mask.columns="mask_"+var_percent_mask.columns
#var_percent_mask=var_percent_mask.truncate(initialdate,enddate)

In [12]:
#added to the generales dataframe and the missing initial values add 0s cause people was not wearing masks at the beggining
exogenas=pd.concat([exogenas,var_percent_mask],axis=1)
exogenas.fillna(0,inplace=True)

In [13]:
#flights per day
#url3 = "https://drive.google.com/file/d/1jLzSGGbQY8OyD1YTNZAJFRmvxy7lA2V_/view?usp=sharing"
#path3 = 'https://drive.google.com/uc?export=download&id='+url3.split('/')[-2]
url3 = 'https://ansperformance.eu/download/xls/Airport_Traffic.xlsx'
flights = pd.read_excel(url3, sheet_name="DATA", date_parser="FLT_DATE", index_col ="FLT_DATE")

In [14]:
flights["STATE_NAME"].unique()

array(['Belgium', 'Germany', 'Estonia', 'Finland', 'United Kingdom',
       'Netherlands', 'Ireland', 'Denmark', 'Luxembourg', 'Norway',
       'Poland', 'Sweden', 'Latvia', 'Lithuania', 'Spain', 'Albania',
       'Bulgaria', 'Cyprus', 'Croatia', 'France', 'Greece', 'Hungary',
       'Italy', 'Slovenia', 'Czech Republic', 'Malta', 'Austria',
       'Portugal', 'Bosnia and Herzegovina', 'Romania', 'Switzerland',
       'Turkey', 'Moldova', 'The former Yugoslav Republic of Macedonia',
       'Serbia', 'Montenegro', 'Slovakia', 'Armenia', 'Georgia',
       'Ukraine', 'Morocco', 'Israel', 'Republic of North Macedonia',
       'North Macedonia'], dtype=object)

In [15]:
for i in flights["STATE_NAME"].unique():
  flights_country=flights[flights["STATE_NAME"]==i]
  flights_country=flights_country.resample("1D").sum()
  flights_country_arr=flights_country['FLT_ARR_1'].truncate(before=initialdate)
  flights_country_arr.rename("FlightsArr_{}".format(i),inplace=True)
  exogenas=pd.concat([exogenas,flights_country_arr],axis=1)

In [18]:
#creating the csv file
exogenas.to_csv('./data/exogenas.csv')

In [19]:
exogenas.loc[:, exogenas.columns.str.contains('mask')]

,mask_Denmark,mask_Finland,mask_France,mask_Germany,mask_Italy,mask_Norway,mask_Spain,mask_Sweden,mask_UK
2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2020-08-09,4.0,7.0,84.0,65.0,81.0,5.0,89.0,6.0,69.0
2020-08-10,4.0,7.0,84.0,65.0,81.0,5.0,89.0,6.0,71.0
2020-08-11,4.0,7.0,84.0,65.0,81.0,5.0,89.0,6.0,73.0
2020-08-12,4.0,7.0,84.0,65.0,81.0,5.0,89.0,6.0,75.0
